In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Model
import tensorflow.keras.optimizers as optimizers
import keras.backend as K

2022-12-10 00:23:16.455230: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from models import createModel

In [3]:
x_train = np.load("./unnorm_no_shuffle_data/x_train.npy")
y_train = np.load("./unnorm_no_shuffle_data/y_train.npy")
x_val = np.load("./unnorm_no_shuffle_data/x_val.npy")
y_val= np.load("./unnorm_no_shuffle_data/y_val.npy")

In [4]:
''' 
Define loss functions
'''

def my_loss(y_true, y_output):
    bce = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
    mse = tf.keras.losses.MeanSquaredError()
    return bce(y_true[:,0:-1], y_output[:,0:-1]) + mse(y_true[:,-1], y_output[:,-1])

def bce_metric(y_true, y_output):
    return K.mean(K.binary_crossentropy(y_true[:,0:-1], y_output[:,0:-1], from_logits=False))

def mse_metric(y_true, y_output):
    return K.mean(K.square(y_true[:,-1] - y_output[:,-1]), axis=-1)
    
def accuracy_metric(y_true, y_output):
    preds = K.cast(K.argmax(y_output[:,0:-1], axis=-1), 'float32')
    return K.mean(K.cast(y_true[:,1] == preds, 'float32'))

# https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
def recall(y_true, y_output):
    preds = K.cast(K.argmax(y_output[:,0:-1], axis=-1), 'float32')
    true_positives = K.sum(K.round(K.clip(y_true[:,1], 0, 1)))
    false_negatives = K.sum(K.abs(preds - 1) * y_true[:,1]) # turns predicted 0 into 1, multiply by predictions
    recall = true_positives / (true_positives + false_negatives  + K.epsilon())
    return recall

# TP / (TP + FP)
def precision(y_true, y_output):
    preds = K.cast(K.argmax(y_output[:,0:-1], axis=-1), 'float32')
    true_positives = K.sum(K.round(K.clip(y_true[:,1], 0, 1)))
    false_positives = K.sum(K.abs(y_true[:,1] - 1) * preds) # turns true y 0 into 1, multiply by predictions
    precision = true_positives / (true_positives + false_positives + K.epsilon())
    return precision

def dice_loss(y_true, y_output):
    # https://datascience.stackexchange.com/questions/66581/is-it-possible-to-make-f1-score-differentiable-and-use-it-directly-as-a-loss-fun
    num = 2*K.sum(y_output[:,1] * y_true[:,1])
    denom = K.sum(y_output[:,1]) + K.sum(y_true[:,1])
    return 1 - (num/denom)
    

def f1_loss(y_true, y_output):
    
    preds = K.cast(K.argmax(y_output[:,0:-1], axis=-1), 'float32')
    true_positives = K.sum(K.round(K.clip(y_true[:,1], 0, 1)))
    false_positives = K.sum((y_true[:,1] - 1) * -preds) # if y=0, p=1, then -1*-1=1
    false_negatives = K.sum((preds - 1) *-y_true[:,1]) # if p=0, y=1, then -1*-1=1
    
    rec = true_positives / (true_positives + false_negatives  + K.epsilon())
    prec = true_positives / (true_positives + false_positives + K.epsilon())
    
    f1 = 2 * (prec * rec) / (prec + rec + K.epsilon())
    
    return 1 - f1

In [5]:
model = createModel()

2022-12-10 00:23:24.655633: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
LEARNING_RATE = 0.000001
BETA_1 = 0.9
BETA_2 = 0.999
EPS = 1e-07

# Better optimizer
lr_schedule = optimizers.schedules.ExponentialDecay(
    initial_learning_rate=LEARNING_RATE,
    decay_steps=10000,
    decay_rate=0.9)

scheduled_opt = optimizers.Adam(
    learning_rate=lr_schedule,
    beta_1=BETA_1,
    beta_2=BETA_2,
    epsilon=EPS)

opt = optimizers.Adam(
    learning_rate=LEARNING_RATE,
    beta_1=BETA_1,
    beta_2=BETA_2,
    epsilon=EPS)

In [7]:
model.compile(loss = dice_loss, optimizer = scheduled_opt, metrics = [accuracy_metric, bce_metric, mse_metric, recall, precision])

In [8]:
NUM_EPOCHS = 3
history = model.fit(x_train[:,:,4:], y_train, epochs=NUM_EPOCHS, batch_size=128, validation_data=(x_val[:,:,4:], y_val))

Epoch 1/3
1537/1537 [==============================] - 6s 3ms/step - loss: 0.8499 - accuracy_metric: 0.1488 - bce_metric: 12.8426 - mse_metric: 18861.4258 - recall: 0.9425 - precision: 0.0876 - val_loss: 0.8859 - val_accuracy_metric: 0.1596 - val_bce_metric: 12.6665 - val_mse_metric: 18702.5566 - val_recall: 0.2580 - val_precision: 0.0829
Epoch 2/3
1537/1537 [==============================] - 4s 2ms/step - loss: 0.8499 - accuracy_metric: 0.1492 - bce_metric: 12.8387 - mse_metric: 18907.5332 - recall: 0.9418 - precision: 0.0877 - val_loss: 0.8859 - val_accuracy_metric: 0.1567 - val_bce_metric: 12.7133 - val_mse_metric: 18717.5039 - val_recall: 0.2588 - val_precision: 0.0826
Epoch 3/3
1537/1537 [==============================] - 4s 3ms/step - loss: 0.8499 - accuracy_metric: 0.1466 - bce_metric: 12.8819 - mse_metric: 18895.5273 - recall: 0.9435 - precision: 0.0874 - val_loss: 0.8859 - val_accuracy_metric: 0.1559 - val_bce_metric: 12.7300 - val_mse_metric: 18722.9766 - val_recall: 0.2592 -

In [50]:
model_string = f"models/dice_checked_data_model/weights_epochs{NUM_EPOCHS}"
model.save_weights(model_string)